# DisSysLab — Track A Jupyter Wizard (Local)

This notebook is for **local Jupyter** development (not Colab). Put this file in the **root of your `DisSysLab/` repo**.

It will:
- Install DisSysLab from a **wheel in `dist/`** if present, otherwise do **`pip install -e .`** from the repo root.
- Run **token-free demos** (uppercase, reverse, word count, number scaling).
- Optionally run a **GPT demo** if `OPENAI_API_KEY` is set, or if `get_credentials.py` provides it.
- Show a **minimal DisSysLab quick-start pipeline** for new users.

If anything fails, the notebook prints actionable hints instead of crashing.

In [64]:
# --- 0) Install DisSysLab (wheel preferred, else editable) ---
import os, glob, sys
from pathlib import Path

repo_root = Path('.').resolve()
if not (repo_root / 'pyproject.toml').exists():
    print('[Setup] ERROR: Please run this notebook from the root of your DisSysLab repository (pyproject.toml not found).')
else:
    wheel_candidates = sorted(glob.glob('dist/*.whl'))
    if wheel_candidates:
        wheel = wheel_candidates[0]
        print(f'[Setup] Installing from wheel: {wheel}')
        !python3 -m pip install -q --upgrade pip
        !python3 -m pip install -q --no-cache-dir "{wheel}"
    else:
        print('[Setup] No wheel found in dist/. Falling back to editable install (pip install -e .).')
        !python3 -m pip install -q --upgrade pip
        !python3 -m pip install -e .

[Setup] Installing from wheel: dist/dsl-0.1.0-py3-none-any.whl


In [65]:
# --- 1) Compatibility imports (handles current and in-flight refactors) ---
import importlib, inspect

def _try_import(path):
    try:
        return importlib.import_module(path)
    except Exception as e:
        return None

# Try new-style paths first, then older paths as fallbacks
mod_gen = _try_import('dsl.block_lib.generators.stream_generators') or _try_import('dsl.block_lib.stream_generators')
mod_xf  = _try_import('dsl.block_lib.transformers.stream_transformers') or _try_import('dsl.block_lib.stream_transformers')
mod_rec = _try_import('dsl.block_lib.recorders.stream_recorders') or _try_import('dsl.block_lib.stream_recorders')
core    = _try_import('dsl.core')

if not core:
    raise ImportError('[Imports] Could not import dsl.core. Make sure installation succeeded.')

from dsl.core import Network

if not (mod_gen and mod_xf and mod_rec):
    raise ImportError('[Imports] Could not import one or more block libraries. Check your package version.')

# Export canonical names
generate = getattr(mod_gen, 'generate')
WrapFunction = getattr(mod_xf, 'WrapFunction', None) or getattr(mod_xf, 'StreamTransformer', None)
PromptToBlock = getattr(mod_xf, 'PromptToBlock', None)

# Recorder
RecordToList = getattr(mod_rec, 'RecordToList', None)
if RecordToList is None:
    # Newer generic record wrapper? Try a graceful fallback adapter
    Record = getattr(mod_rec, 'RecordToList', None) or getattr(mod_rec, 'Record', None)
    if Record is None:
        raise ImportError('[Imports] Neither RecordToList nor Record found in recorders module.')
    else:
        RecordToList = Record  # assume same signature

print('[Imports] OK: generate, WrapFunction/StreamTransformer, PromptToBlock (optional), RecordToList')

[Imports] OK: generate, WrapFunction/StreamTransformer, PromptToBlock (optional), RecordToList


In [66]:
# --- 2) Tiny helpers for compatibility with input_key/output_key kwargs ---
import functools

def make_transform(fn, input_key=None, output_key=None):
    """Create a transformer block with best-effort support for input/output keys.
    Falls back gracefully if the underlying class does not accept those kwargs.
    """
    if WrapFunction is None:
        raise RuntimeError('WrapFunction/StreamTransformer not found.')
    try:
        # Try the modern signature
        return WrapFunction(fn, input_key=input_key, output_key=output_key)
    except TypeError:
        # Fallback to simple call
        return WrapFunction(fn)

def print_if_available(net):
    for name in (
        'print_connections_only',
        'print_parameter_mappings_only',
        'draw',
    ):
        if hasattr(net, name):
            try:
                print(f'[Viz] Calling net.{name}()')
                getattr(net, name)()
            except Exception as e:
                print(f'[Viz] {name} failed: {e}')

## Token‑free Demos
These demos run without any API keys.

In [67]:
# --- 3) Demo A: Uppercase ---
results = []
net = Network(
    blocks={
        'gen': generate(['hello', 'distributed', 'systems']),
        'xf': make_transform(lambda s: s.upper()),
        'rec': RecordToList(results),
    },
    connections=[('gen','out','xf','in'),('xf','out','rec','in')]
)
net.compile_and_run()
print('Uppercase results:', results)
print_if_available(net)

[StreamTransformer.handle_msg] Received: hello
[StreamTransformer.handle_msg] Received: distributed
[StreamTransformer.handle_msg] Received: systems
Uppercase results: ['HELLO', 'DISTRIBUTED', 'SYSTEMS']


In [68]:
# --- 4) Demo B: Reverse strings ---
results = []
net = Network(
    blocks={
        'gen': generate(['alpha', 'beta', 'gamma']),
        'xf': make_transform(lambda s: s[::-1]),
        'rec': RecordToList(results),
    },
    connections=[('gen','out','xf','in'),('xf','out','rec','in')]
)
net.compile_and_run()
print('Reverse results:', results)

[StreamTransformer.handle_msg] Received: alpha
[StreamTransformer.handle_msg] Received: beta
[StreamTransformer.handle_msg] Received: gamma
Reverse results: ['ahpla', 'ateb', 'ammag']


In [69]:
# --- 5) Demo C: Word count ---
def word_count(s: str) -> int:
    return len(s.split())

results = []
net = Network(
    blocks={
        'gen': generate(['one two', 'three four five', 'six']),
        'xf': make_transform(word_count),
        'rec': RecordToList(results),
    },
    connections=[('gen','out','xf','in'),('xf','out','rec','in')]
)
net.compile_and_run()
print('Word-count results:', results)

[StreamTransformer.handle_msg] Received: one two
[StreamTransformer.handle_msg] Received: three four five
[StreamTransformer.handle_msg] Received: six
Word-count results: [2, 3, 1]


In [70]:
# --- 6) Demo D: Number scaling ---
def scale(nums, factor=10):
    return [x * factor for x in nums]

results = []
net = Network(
    blocks={
        'gen': generate([[1,2,3],[4,5],[6]]),
        'xf': make_transform(lambda arr: scale(arr, factor=3)),
        'rec': RecordToList(results),
    },
    connections=[('gen','out','xf','in'),('xf','out','rec','in')]
)
net.compile_and_run()
print('Scaled-number results:', results)

[StreamTransformer.handle_msg] Received: [1, 2, 3]
[StreamTransformer.handle_msg] Received: [4, 5]
[StreamTransformer.handle_msg] Received: [6]
Scaled-number results: [[3, 6, 9], [12, 15], [18]]


## Optional: GPT Demo
This cell tries to read `OPENAI_API_KEY` from your environment. If not set, it will try to import `get_credentials.py` from the repo root and read `OPENAI_API_KEY` from there. If no key is found, the cell will skip the GPT example.

In [10]:
import inspect

def make_prompt_block(PromptToBlock, messages):
    """
    Create a PromptToBlock regardless of which kw name your version expects.
    Tries: messages, prompt, template, system_prompt/user_template pairs.
    """
    sig = inspect.signature(PromptToBlock)
    params = set(sig.parameters.keys())

    # Common single-arg names
    for kw in ("messages", "prompt", "template", "prompt_messages"):
        if kw in params:
            return PromptToBlock(**{kw: messages})

    # Pair-style constructors (system/user)
    if {"system_prompt", "user_template"} <= params:
        # Split messages into system + user if possible; else make a simple pair
        system = "You are a helpful assistant."
        user = "{text}"
        for m in messages:
            if m.get("role") == "system":
                system = m.get("content", system)
            if m.get("role") == "user":
                user = m.get("content", user)
        return PromptToBlock(system_prompt=system, user_template=user)

    # Fallback: raise with helpful info
    raise TypeError(
        f"Don't know how to init PromptToBlock with params {sorted(params)}. "
        "Try renaming 'messages' to whatever your class expects."
    )

# Show the constructor so we can see what it wants
from dsl.block_lib.stream_transformers import PromptToBlock
print("[Debug] PromptToBlock signature:", inspect.signature(PromptToBlock))

[Debug] PromptToBlock signature: (prompt: str, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, name: Optional[str] = None)


In [ ]:
import os, sys
sys.path.append(os.getcwd())  # ensure repo root is on sys.path

from dsl.utils import get_credentials

key = ""
# Prefer a direct key if the module exposes one (future-proof), else use the helper
if hasattr(get_credentials, "OPENAI_API_KEY"):
    key = get_credentials.OPENAI_API_KEY
elif hasattr(get_credentials, "get_openai_key"):
    key = get_credentials.get_openai_key()

if key:
    os.environ["OPENAI_API_KEY"] = key
    print("[GPT] OPENAI_API_KEY loaded from dsl/utils/get_credentials.py")
else:
    print("[GPT] No key found in get_credentials. Skipping GPT demo.")


if not PromptToBlock:
    print('[GPT] PromptToBlock not available in your current version. Skipping GPT demo.')
elif not key:
    print('[GPT] No OPENAI_API_KEY found. Skipping GPT demo.')
else:
    try:
        from dsl.block_lib.prompt_templates import sentiment_prompt

        results = []
        net = Network(
            blocks={
                'gen': generate(['I love distributed systems!', 'I hate bugs.']),
                'gpt': PromptToBlock(sentiment_prompt()),
                'rec': RecordToList(results),
            },
            connections=[('gen','out','gpt','in'),('gpt','out','rec','in')]
        )
        net.compile_and_run()
        print('[GPT] Results:', results)
    except Exception as e:
        print('[GPT] Demo failed:', e)

[GPT] OPENAI_API_KEY loaded from dsl/utils/get_credentials.py


❌ PromptToBlock error: 'function' object has no attribute 'replace'

❌ PromptToBlock error: 'function' object has no attribute 'replace'

[GPT] Results: []


## Minimal Quick‑Start Pipeline
A small, readable example you can show to first‑timers. If your framework supports dict messages with a `data` key, we try to use it; otherwise we fall back to plain strings.

In [ ]:
# --- 8) Quick‑Start: Dict messages if supported ---
results = []

def to_upper_data(msg):
    # If dict-based messages are used, expect msg['data']; else treat msg as a string
    if isinstance(msg, dict) and 'data' in msg:
        return {**msg, 'data': str(msg['data']).upper()}
    return str(msg).upper()

try:
    # Prefer explicit keys when the transformer supports input_key/output_key
    xf = make_transform(to_upper_data, input_key='data', output_key='data')
except Exception:
    # Fallback: no key routing
    xf = make_transform(to_upper_data)

net = Network(
    blocks={
        'gen': generate([{'data': 'hello'}, {'data': 'world'}]),
        'xf': xf,
        'rec': RecordToList(results),
    },
    connections=[('gen','out','xf','in'),('xf','out','rec','in')]
)
net.compile_and_run()
print('Quick‑start results:', results)